In [96]:
import pandas as pd

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Disable horizontal wrapping

In [97]:
import inflection

def rename_columns_to_snake_case(df):
    new_column_names = [inflection.underscore(col) for col in df.columns]
    column_mapping = dict(zip(df.columns, new_column_names))
    return df.rename(columns=column_mapping)

In [98]:
import re
import numpy as np

percentage_pattern = r'^[-+]?(\d+(\.\d*)?|\.\d+)%$'

def is_percentage_string(cell_value):
    return isinstance(cell_value, str) and re.match(percentage_pattern, cell_value) is not None

# Function to get the percentage string columns of a DataFrame
def get_percentage_string_columns(df):
    # Filter the columns based on the pattern matching
    first_row = df.truncate(after=0)
    values_are_percentage_strings = first_row.applymap(is_percentage_string)
    return list(values_are_percentage_strings.columns[values_are_percentage_strings.iloc[0]])    

# Function to convert percentage string columns to floats 0.0 to 1.0
# Example usage:
# Convert the percentage string columns of 'df' to floats ranging from 0.0 to 1.0
# df = convert_percentage_string_columns(df)
def convert_percentage_string_columns(df):
    # Get the percentage string columns
    percentage_string_columns = get_percentage_string_columns(df)
    
    # Convert the percentage strings to floats
    df[percentage_string_columns] = df[percentage_string_columns].apply(lambda x: x.str.rstrip('%').astype(float) / 100)
    
    return df

def convert_star_cells_to_nan(df):
    return df.replace("*", np.nan)

In [99]:
import json
import pandas as pd

def normalize_json_column(df, column):
    normalized_json = pd.json_normalize(df[column].to_list())
    return pd.concat([df, normalized_json], axis="columns").drop(column, axis="columns")

In [100]:
import re

percentage_pattern = r'^[-+]?(\d+(\.\d*)?|\.\d+)%$'

def is_percentage_string(cell_value):
    return isinstance(cell_value, str) and re.match(percentage_pattern, cell_value) is not None

# Function to get the percentage string columns of a DataFrame
def get_percentage_string_columns(df):
    # Filter the columns based on the pattern matching
    first_row = df.truncate(after=0)
    values_are_percentage_strings = first_row.applymap(is_percentage_string)
    return list(values_are_percentage_strings.columns[values_are_percentage_strings.iloc[0]])

# Function to convert percentage string columns to floats 0.0 to 1.0
# Example usage:
# Convert the percentage string columns of 'df' to floats ranging from 0.0 to 1.0
# df = convert_percentage_string_columns(df)
def convert_percentage_string_columns(df):
    # Get the percentage string columns
    percentage_string_columns = get_percentage_string_columns(df)
    
    # Convert the percentage strings to floats
    df[percentage_string_columns] = df[percentage_string_columns].apply(lambda x: x.str.rstrip('%').astype(float))
    
    return df

In [101]:
from data import get_all_data_by_ward
ward_raw_data = get_all_data_by_ward()
ward_data = rename_columns_to_snake_case(ward_raw_data)
ward_data = normalize_json_column(ward_data, "votes")
ward_data


/Users/alex/Desktop/camera-obscura/venv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,ward_no,geometry,electorate,total ballot papers,ballot papers rejected: no readable uim,ballot papers rejected: no first preference,ballot papers rejected: more than one first preference,ballot papers rejected: identifies the voter,ballot papers rejected: unmarked or void for uncertainty,total ballot papers rejected,total ballot papers included in count,name,Independent,Women's Equality Party,Scottish Labour Party,Scottish Liberal Democrats,Scottish Green Party,Scottish Trade Unionist and Socialist Coalition,Independence for Scotland Party,Alba Party,Scottish Libertarian Party,Socialist Labour Party,Sovereignty,West Dunbartonshire Community Party,Scottish National Party,Scottish Conservative and Unionist Party,Scottish Family Party,"Freedom Alliance- Integrity, Society, Economy",UK Independence Party,Scottish Socialist Party,The Rubbish Party,The Pensioner's Party,Social Democratic Party,Volt UK,British Unionist Party,Vanguard Party,Scottish Eco-Federalist Party,Workers Party of Britain,Communist Party of Britain
0,8,"POLYGON ((-3.22292 55.91343, -3.22255 55.91328...",19454,11540,0,29,94,1,2,126,11414,Colinton/Fairmilehead,0,0,3812,1416,621,0,0,0,0,0,0,0,1969,3417,179,0,0,0,0,0,0,0,0,0,0,0,0
1,17,"POLYGON ((-3.07495 55.94957, -3.07929 55.94509...",26118,11426,0,9,215,0,19,243,11183,Portobello/Craigmillar,182,0,2677,480,1808,0,0,142,0,0,0,0,4182,1712,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,"POLYGON ((-3.21639 55.94273, -3.21705 55.94219...",23850,8557,0,4,158,0,20,182,8375,Sighthill/Gorgie,797,0,1929,354,1116,0,0,164,0,0,0,0,3029,986,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,"POLYGON ((-3.31706 55.92216, -3.31501 55.92282...",24312,11266,0,19,158,0,2,179,11087,Pentland Hills,0,0,1896,885,870,0,0,0,32,0,0,0,3475,3770,159,0,0,0,0,0,0,0,0,0,0,0,0
4,16,"POLYGON ((-3.11686 55.91472, -3.11727 55.91381...",28977,12665,0,28,201,0,3,232,12433,Liberton/Gilmerton,0,0,4144,507,1023,0,0,222,0,0,0,0,3816,2239,187,0,0,295,0,0,0,0,0,0,0,0,0
5,9,"POLYGON ((-3.20552 55.94331, -3.20647 55.94272...",18284,9112,0,9,46,1,28,84,9028,Fountainbridge/Craiglockhart,0,0,2026,642,1800,0,0,0,41,0,0,0,2051,2399,69,0,0,0,0,0,0,0,0,0,0,0,0
6,10,"POLYGON ((-3.18427 55.91896, -3.18457 55.91801...",25204,13618,0,18,49,0,4,71,13547,Morningside,0,0,3145,2877,2854,0,0,132,52,0,0,0,2097,2390,0,0,0,0,0,0,0,0,0,0,0,0,0
7,5,"POLYGON ((-3.19278 55.97028, -3.19674 55.96782...",24608,13542,0,7,91,0,28,126,13416,Inverleith,17,0,1684,4522,1714,0,0,0,53,0,0,0,2641,2689,96,0,0,0,0,0,0,0,0,0,0,0,0
8,4,"POLYGON ((-3.19730 55.98163, -3.19740 55.98088...",24934,10733,0,34,135,0,9,178,10555,Forth,0,130,1995,2077,1337,0,0,0,0,0,0,0,3270,1594,152,0,0,0,0,0,0,0,0,0,0,0,0
9,11,"POLYGON ((-3.16747 55.95728, -3.16725 55.95687...",23510,8938,0,1,55,1,24,81,8857,City Centre,297,0,1297,1341,1755,0,0,105,0,0,0,0,2166,1896,0,0,0,0,0,0,0,0,0,0,0,0,0


In [106]:
import geopandas as gpd
import pandas as pd

simd_raw_shp = gpd.read_file("../data/simd2020_withgeog/sc_dz_11.shp").to_crs(ward_data.crs)
simd_raw_csv = pd.read_csv("../data/simd2020_withgeog/simd2020_withinds.csv")

simd_data = simd_raw_shp.merge(simd_raw_csv, left_on="DataZone", right_on="Data_Zone", how="left").drop("Data_Zone", axis="columns")
simd_data = rename_columns_to_snake_case(simd_data)
simd_data = simd_data.rename(
    columns={"employment_rate": "employment_deprived_rate",
             "income_rate": "income_deprived_rate"}
) # simd oddly names it as such
simd_data = convert_star_cells_to_nan(simd_data)
simd_data = convert_percentage_string_columns(simd_data)
simd_data = simd_data[simd_data.council_area == "City of Edinburgh"]

simd_data

,data_zone,name,tot_pop2011,res_pop2011,hh_cnt2011,std_area_ha,std_area_km2,shape_leng,shape_area,geometry,intermediate_zone,council_area,total_population,working_age_population,simd2020v2_rank,simd_2020v2_percentile,simd2020v2_vigintile,simd2020v2_decile,simd2020v2_quintile,simd2020v2_income_domain_rank,simd2020_employment_domain_rank,simd2020_health_domain_rank,simd2020_education_domain_rank,simd2020_access_domain_rank,simd2020_crime_domain_rank,simd2020_housing_domain_rank,income_deprived_rate,income_count,emloyment_deprived_rate,employment_count,cif,alcohol,drug,smr,depress,lbwt,emerg,attendance,attainment,no_qualifications,not_participating,university,crime_count,crime_rate,overcrowded_count,nocentralheating_count,overcrowded_rate,nocentralheating_rate,drive_petrol,drive_gp,drive_post,drive_primary,drive_retail,drive_secondary,pt_gp,pt_post,pt_retail,broadband
1911,S01008417,Balerno and Bonnington Village - 01,726,726,299,1029.993186,10.299931,20191.721420,1.029993e+07,"POLYGON ((-3.35779 55.88156, -3.35752 55.88116...",Balerno and Bonnington Village,City of Edinburgh,708,397,5537,80,16,8,4,5218.5,5564.0,6653,6691,853,5392.0,6350.0,5.0,32,3.0,13,35.0,10.0,0.0,26.0,14.0,7.0,56.0,92.0,6.14,28.0,0.0,30.0,6,86,17,8,2.0,1.0,3.147979,7.470992,3.602979,5.257358,3.522979,4.135769,22.373316,16.205441,16.552056,15.0
1912,S01008418,Balerno and Bonnington Village - 02,692,692,312,2357.049745,23.570499,25944.861787,2.357050e+07,"POLYGON ((-3.31542 55.88531, -3.30846 55.88127...",Balerno and Bonnington Village,City of Edinburgh,691,378,6119,88,18,9,5,6236.5,6386.5,6440,6733,988,5063.0,6650.0,2.0,17,2.0,8,30.0,57.0,50.0,38.0,12.0,4.0,72.0,94.0,6.30,24.0,0.0,13.0,7,103,7,10,1.0,1.0,3.582608,7.658158,4.036864,3.559743,3.956525,4.767980,17.633049,12.754447,13.100199,7.0
1913,S01008419,Balerno and Bonnington Village - 03,525,500,193,136.645688,1.366457,5738.935106,1.366457e+06,"POLYGON ((-3.33131 55.88937, -3.33154 55.88922...",Balerno and Bonnington Village,City of Edinburgh,541,338,5200,75,15,8,4,4861.0,4315.0,5417,6907,2204,3594.0,5372.5,5.0,29,6.0,19,60.0,57.0,0.0,100.0,11.0,0.0,72.0,NaN,6.60,34.0,0.0,25.0,10,187,30,0,6.0,0.0,2.436124,6.391605,2.890733,3.023231,2.810733,3.161976,18.665124,13.401943,13.749981,0.0
1914,S01008420,Balerno and Bonnington Village - 04,476,476,190,10.519669,0.105197,1890.096713,1.051967e+05,"POLYGON ((-3.33842 55.88098, -3.33823 55.88070...",Balerno and Bonnington Village,City of Edinburgh,466,274,6431,93,19,10,5,6298.0,5980.0,5913,6262,2702,4957.0,6957.0,2.0,11,3.0,8,35.0,8.0,37.0,80.0,15.0,0.0,97.0,NaN,5.62,30.0,0.0,13.0,5,109,2,0,0.0,0.0,2.318151,6.596964,3.007096,2.155458,2.926565,3.369638,16.614664,10.839664,11.187521,0.0
1915,S01008421,Balerno and Bonnington Village - 05,858,858,394,14.654407,0.146542,2645.609037,1.465441e+05,"POLYGON ((-3.33787 55.88523, -3.33768 55.88492...",Balerno and Bonnington Village,City of Edinburgh,884,537,3441,50,10,5,3,3330.5,2861.0,2866,3907,4721,2575.0,4598.0,10.0,91,10.0,51,105.0,99.0,65.0,94.0,21.0,7.0,103.0,88.0,5.17,88.0,2.0,11.0,23,264,61,6,7.0,1.0,1.305804,6.020373,2.037905,2.588223,1.959213,2.917302,16.158549,6.965932,7.310676,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,S01009009,Queensferry West - 01,476,476,252,21.039352,0.210393,2484.606370,2.103935e+05,"POLYGON ((-3.40676 55.99353, -3.40655 55.99343...",Queensferry West,City of Edinburgh,476,287,1664,24,5,3,2,1506.0,1584.0,1888,1836,5485,889.0,1634.0,20.0,93,14.0,41,135.0,153.0,189.0,100.0,22.0,4.0,115.0,76.0,4.88,118.0,3.0,6.0,25,532,73,9,15.0,2.0,2.265861,1.986765,2.028529,2.428171,2.760699,5.707500,5.651681,5.651681,7.509002,0.0
2504,S01009010,Queensferry West - 02,1021,1021,404,20.571212,0.205711,2549.606615,2.057121e+05,"POLYGON ((-3.40947 55.98996, -3.40948 55.98955...",Queensferry West,City of Edinburgh,977,6

In [108]:
import geopandas as gpd

ward_simd_data = gpd.overlay(simd_data, ward_data, how="intersection")
ward_simd_data = ward_simd_data.drop_duplicates(subset=["data_zone"], keep=False)
ward_simd_data

,data_zone,name_1,tot_pop2011,res_pop2011,hh_cnt2011,std_area_ha,std_area_km2,shape_leng,shape_area,intermediate_zone,council_area,total_population,working_age_population,simd2020v2_rank,simd_2020v2_percentile,simd2020v2_vigintile,simd2020v2_decile,simd2020v2_quintile,simd2020v2_income_domain_rank,simd2020_employment_domain_rank,simd2020_health_domain_rank,simd2020_education_domain_rank,simd2020_access_domain_rank,simd2020_crime_domain_rank,simd2020_housing_domain_rank,income_deprived_rate,income_count,emloyment_deprived_rate,employment_count,cif,alcohol,drug,smr,depress,lbwt,emerg,attendance,attainment,no_qualifications,not_participating,university,crime_count,crime_rate,overcrowded_count,nocentralheating_count,overcrowded_rate,nocentralheating_rate,drive_petrol,drive_gp,drive_post,drive_primary,drive_retail,drive_secondary,pt_gp,pt_post,pt_retail,broadband,ward_no,electorate,total ballot papers,ballot papers rejected: no readable uim,ballot papers rejected: no first preference,ballot papers rejected: more than one first preference,ballot papers rejected: identifies the voter,ballot papers rejected: unmarked or void for uncertainty,total ballot papers rejected,total ballot papers included in count,name_2,Independent,Women's Equality Party,Scottish Labour Party,Scottish Liberal Democrats,Scottish Green Party,Scottish Trade Unionist and Socialist Coalition,Independence for Scotland Party,Alba Party,Scottish Libertarian Party,Socialist Labour Party,Sovereignty,West Dunbartonshire Community Party,Scottish National Party,Scottish Conservative and Unionist Party,Scottish Family Party,"Freedom Alliance- Integrity, Society, Economy",UK Independence Party,Scottish Socialist Party,The Rubbish Party,The Pensioner's Party,Social Democratic Party,Volt UK,British Unionist Party,Vanguard Party,Scottish Eco-Federalist Party,Workers Party of Britain,Communist Party of Britain,geometry
0,S01008417,Balerno and Bonnington Village - 01,726,726,299,1029.993186,10.299931,20191.721420,1.029993e+07,Balerno and Bonnington Village,City of Edinburgh,708,397,5537,80,16,8,4,5218.5,5564.0,6653,6691,853,5392.0,6350.0,5.0,32,3.0,13,35.0,10.0,0.0,26.0,14.0,7.0,56.0,92.0,6.14,28.0,0.0,30.0,6,86,17,8,2.0,1.0,3.147979,7.470992,3.602979,5.257358,3.522979,4.135769,22.373316,16.205441,16.552056,15.0,2,24312,11266,0,19,158,0,2,179,11087,Pentland Hills,0,0,1896,885,870,0,0,0,32,0,0,0,3475,3770,159,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-3.35752 55.88116, -3.35685 55.88126..."
1,S01008418,Balerno and Bonnington Village - 02,692,692,312,2357.049745,23.570499,25944.861787,2.357050e+07,Balerno and Bonnington Village,City of Edinburgh,691,378,6119,88,18,9,5,6236.5,6386.5,6440,6733,988,5063.0,6650.0,2.0,17,2.0,8,30.0,57.0,50.0,38.0,12.0,4.0,72.0,94.0,6.30,24.0,0.0,13.0,7,103,7,10,1.0,1.0,3.582608,7.658158,4.036864,3.559743,3.956525,4.767980,17.633049,12.754447,13.100199,7.0,2,24312,11266,0,19,158,0,2,179,11087,Pentland Hills,0,0,1896,885,870,0,0,0,32,0,0,0,3475,3770,159,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-3.30846 55.88127, -3.30440 55.88040..."
2,S01008419,Balerno and Bonnington Village - 03,525,500,193,136.645688,1.366457,5738.935106,1.366457e+06,Balerno and Bonnington Village,City of Edinburgh,541,338,5200,75,15,8,4,4861.0,4315.0,5417,6907,2204,3594.0,5372.5,5.0,29,6.0,19,60.0,57.0,0.0,100.0,11.0,0.0,72.0,NaN,6.60,34.0,0.0,25.0,10,187,30,0,6.0,0.0,2.436124,6.391605,2.890733,3.023231,2.810733,3.161976,18.665124,13.401943,13.749981,0.0,2,24312,11266,0,19,158,0,2,179,11087,Pentland Hills,0,0,1896,885,870,0,0,0,32,0,0,0,3475,3770,159,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-3.33154 55.88922, -3.33107 55.88893..."
3,S01008420,Balerno and Bonnington Village - 04,476,476,190,10.519669,0.105197,1890.096713,1.051967e+05,Balerno and Bonnington Village,City of Edinburgh,466,274,6431,93,19,10,5,6298.0,5980.0,5913,6262,2702,4957.0,6957.0,2.0,11,3.0,8,35.0,8.0,37.0,80.0,15.0,0.0,97.0,NaN,5.62,30.0,0.0,13.0,5,109,2,0,0.0,0.0,2.318151,6.596964,3.007096,2.155458,2.926565,3.369638,16.614664,10.839

In [107]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have the 'intersection' GeoDataFrame with the desired columns

# Extract the party names from the 'votes' column
parties = ["Scottish National Party", "Scottish Conservative and Unionist Party", "Scottish Labour Party", "Scottish Green Party", "Scottish Liberal Democrats"]
columns_of_interest = ["employment_deprived_rate", "university", "crime_rate", "working_age_population", "depress", "drive_gp", "income_deprived_rate"]

# Create an empty DataFrame to store the correlations
correlation_columns = columns_of_interest + parties
correlation_df = ward_simd_data[correlation_columns].corr()

# Create a heatmap of the correlation table
sns.heatmap(correlation_df, annot=True, cmap='RdYlGn', fmt=".2f")
plt.title('Correlation between Party Votes and SIMD Columns')
plt.xlabel('Party')
plt.ylabel('SIMD Columns')
plt.show()


KeyError: "['employment_deprived_rate'] not in index"